# Poles & Zeros: from the **S-domain** to the **Z-domain** (with Bode + impulse response)

This notebook is a guided, hands-on tour of:

- what the **S-domain** (continuous-time / analog) and **Z-domain** (discrete-time / digital) mean
- how to take **analog poles & zeros** (given in rad/s) and map them into the **z-plane**
- how to interpret the **frequency response** from **geometry on the unit circle**
- how to compute and plot:
  - **s-plane** pole/zero plot
  - **z-plane** pole/zero plot **with the unit circle**
  - **frequency response** (magnitude + phase)
  - **impulse response** (time domain)

We use the Chaparral M25 example poles/zeros:

```python
poles = [-1190, -0.157+3e-6j, -0.157 ]
zeros = [0, 0, 0, -4080]
K = 0.2917
```

> Note: these poles/zeros are *analog* (s-plane), typically expressed in **rad/s**.


## 0. Setup

We'll use `numpy`, `scipy.signal`, and `matplotlib`.

- Poles/zeros are in the **s-plane**: \( s = \sigma + i\Omega \) (units: rad/s)
- For a discrete-time system, we work in the **z-plane**: \( z \) (dimensionless)
- A standard mapping from analog to digital is the **matched pole-zero** map:

\[
z = e^{s\Delta t}
\]

where \(\Delta t = 1/f_s\) is the sampling interval.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

## 1. The S-domain (continuous-time) in one picture

In continuous time, many LTI systems can be described by a transfer function:

\[
H(s) = K \frac{\prod_k (s - z_k)}{\prod_m (s - p_m)}
\]

- **Poles** (\(p_m\)) are where the denominator goes to zero → they govern **stability** and **time constants**.
- **Zeros** (\(z_k\)) are where the numerator goes to zero → they create **notches**, DC rejection, and shaping.

A physically-realizable stable analog system has poles with **negative real parts** (\(\Re(p_m) < 0\)).

### Chaparral example: define the analog poles/zeros


In [ ]:
# --- Chaparral M25 poles/zeros (analog) ---
poles_s = np.array([-1190, -0.157 + 3e-6j, -0.157], dtype=complex)
zeros_s = np.array([0, 0, 0, -4080], dtype=complex)
K = 0.2917

poles_s, zeros_s

### Plot the s-plane poles & zeros

These are points in the complex **s-plane** (units: rad/s).

> The tiny imaginary part (3e-6 rad/s) corresponds to a period of ~24 days, so it's effectively a real pole in practice.


In [ ]:
def plot_pz_s_plane(poles, zeros, title="Pole-Zero Plot (s-plane)"):
    fig, ax = plt.subplots(figsize=(7, 6))

    if len(zeros):
        ax.scatter(np.real(zeros), np.imag(zeros), marker='o', s=90,
                   facecolors='none', edgecolors='k', label='Zeros')
    if len(poles):
        ax.scatter(np.real(poles), np.imag(poles), marker='x', s=90,
                   color='k', label='Poles')

    ax.axhline(0, linewidth=1)
    ax.axvline(0, linewidth=1)
    ax.set_xlabel("Real(s) [rad/s]")
    ax.set_ylabel("Imag(s) [rad/s]")
    ax.set_title(title)
    ax.grid(True, which="both")
    ax.legend()
    ax.set_aspect('equal', adjustable='datalim')
    plt.tight_layout()
    plt.show()

plot_pz_s_plane(poles_s, zeros_s)

## 2. The Z-domain (discrete-time) and the unit circle

In discrete time, the transfer function is written as:

\[
H(z) = K \frac{\prod_k (z - z_k)}{\prod_m (z - p_m)}
\]

The **frequency response** is obtained by evaluating on the **unit circle**:

\[
z = e^{i\omega}
\quad\Rightarrow\quad
H(e^{i\omega})
\]

where \(\omega\) is digital rad/sample.

### Frequency ↔ angle mapping
If your sampling rate is \(f_s\) (Hz), then a physical frequency \(f\) corresponds to angle:

\[
\theta = 2\pi \frac{f}{f_s}
\]

- DC (0 Hz): \(\theta = 0\) → point \(z=1\)
- Nyquist (\(f_s/2\)): \(\theta = \pi\) → point \(z=-1\)

So as you sweep frequency, you literally **walk around the unit circle**.


## 3. Mapping analog poles/zeros into the z-plane

We will use the **matched pole-zero mapping**:

\[
z = e^{s\Delta t}
\]

This mapping has a nice property:
- Stable analog poles (\(\Re(s)<0\)) map **inside** the unit circle (\(|z|<1\)).

We'll also show an alternative mapping (**bilinear transform**) for comparison:

\[
z = \frac{1 + s\Delta t/2}{1 - s\Delta t/2}
\]

### Choose a sampling rate
Set this to your actual sampling rate.


In [ ]:
fs = 1000.0  # Hz (edit me)
dt = 1.0 / fs
dt

In [ ]:
def s_to_z(points_s, fs, method="matched_z"):
    dt = 1.0 / fs
    points_s = np.asarray(points_s, dtype=complex)
    if method == "matched_z":
        return np.exp(points_s * dt)
    elif method == "bilinear":
        return (1 + points_s*dt/2) / (1 - points_s*dt/2)
    else:
        raise ValueError("method must be 'matched_z' or 'bilinear'")

method = "matched_z"  # or "bilinear"
poles_z = s_to_z(poles_s, fs, method=method)
zeros_z = s_to_z(zeros_s, fs, method=method)

poles_z, zeros_z

### Plot the z-plane poles/zeros + the unit circle

In [ ]:
def plot_pz_z_plane(poles_z, zeros_z, fs, method, title=None):
    fig, ax = plt.subplots(figsize=(7, 6))

    # Unit circle
    th = np.linspace(0, 2*np.pi, 800)
    ax.plot(np.cos(th), np.sin(th), linewidth=1, label="Unit circle")

    if len(zeros_z):
        ax.scatter(np.real(zeros_z), np.imag(zeros_z), marker='o', s=90,
                   facecolors='none', edgecolors='k', label='Zeros (z-plane)')
    if len(poles_z):
        ax.scatter(np.real(poles_z), np.imag(poles_z), marker='x', s=90,
                   color='k', label='Poles (z-plane)')

    ax.axhline(0, linewidth=1)
    ax.axvline(0, linewidth=1)
    ax.set_xlabel("Real(z)")
    ax.set_ylabel("Imag(z)")
    if title is None:
        title = f"Pole-Zero Plot (z-plane), fs={fs:g} Hz, mapping={method}"
    ax.set_title(title)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.grid(True, which="both")
    ax.legend()
    plt.tight_layout()
    plt.show()

plot_pz_z_plane(poles_z, zeros_z, fs=fs, method=method)

## 4. Frequency response from z-plane geometry

For a digital filter:

\[
H(z) = K \frac{\prod_k (z - z_k)}{\prod_m (z - p_m)}
\]

Evaluate on the unit circle \(z=e^{i\omega}\):

\[
|H(e^{i\omega})|
=
|K|\;
\frac{\prod_k |e^{i\omega} - z_k|}
{\prod_m |e^{i\omega} - p_m|}
\]

### The geometric interpretation
At each frequency \(\omega\):

- Compute the distance from the point \(e^{i\omega}\) on the unit circle to **each zero**.
- Multiply those distances together → numerator magnitude contribution.
- Compute the distance from \(e^{i\omega}\) to **each pole**.
- Multiply those distances together → denominator magnitude contribution.

So:
- **Zeros near the unit circle** at a given angle suppress the magnitude (distance small → numerator small).
- **Poles near the unit circle** boost the magnitude (distance small → denominator small).

> In words: **zeros repel**, **poles attract**.


### Mini-demo: one pole + one zero

We’ll draw:
- one pole (×)
- one zero (○)
- the unit circle
- a point on the unit circle corresponding to some frequency

Then we’ll compute the magnitude \(|H(e^{i\omega})|\) as the ratio of distances.


In [ ]:
# A simple example system in z-plane
z0 = 0.8 * np.exp(1j*np.deg2rad(40))    # zero near the unit circle at 40°
p0 = 0.6 * np.exp(1j*np.deg2rad(140))   # pole inside at 140°
K_ex = 1.0

# Choose an evaluation angle (frequency)
theta = np.deg2rad(40)
z_eval = np.exp(1j*theta)

# Distances
d_zero = np.abs(z_eval - z0)
d_pole = np.abs(z_eval - p0)

H_mag_geom = np.abs(K_ex) * d_zero / d_pole
d_zero, d_pole, H_mag_geom

In [ ]:
def plot_one_pole_one_zero(z0, p0, theta):
    fig, ax = plt.subplots(figsize=(7, 6))

    # Unit circle
    th = np.linspace(0, 2*np.pi, 800)
    ax.plot(np.cos(th), np.sin(th), linewidth=1)

    # Points
    ax.scatter(np.real(z0), np.imag(z0), marker='o', s=120,
               facecolors='none', edgecolors='k', label='Zero')
    ax.scatter(np.real(p0), np.imag(p0), marker='x', s=120, color='k', label='Pole')

    z_eval = np.exp(1j*theta)
    ax.scatter(np.real(z_eval), np.imag(z_eval), marker='.', s=220, color='k',
               label='Eval point $e^{i\omega}$')

    # Distance lines
    ax.plot([np.real(z_eval), np.real(z0)], [np.imag(z_eval), np.imag(z0)],
            linewidth=2, label='Distance to zero')
    ax.plot([np.real(z_eval), np.real(p0)], [np.imag(z_eval), np.imag(p0)],
            linewidth=2, label='Distance to pole')

    ax.axhline(0, linewidth=1)
    ax.axvline(0, linewidth=1)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.grid(True, which="both")
    ax.set_title("Geometry of |H(e^{iω})| for one zero and one pole")
    ax.legend(loc="best")
    plt.tight_layout()
    plt.show()

plot_one_pole_one_zero(z0, p0, theta)

## 5. Frequency response for the mapped Chaparral response (Hz)

We’ll build a discrete-time ZPK system from the mapped poles/zeros and compute:

- \(H(e^{i\omega})\) on the unit circle
- plot against **Hz** (cycles/second)

We’ll also draw a vertical line at **400 Hz** (the manual’s “not valid above” guideline).


In [ ]:
# Build discrete-time system in z-domain
sys_z = signal.ZerosPolesGain(zeros_z, poles_z, K, dt=dt)  # dt makes it discrete-time

# Frequency grid in Hz
f = np.logspace(-2, np.log10(fs/2), 2000)  # from 0.01 Hz to Nyquist
w_digital = 2*np.pi * f / fs  # rad/sample

# Evaluate frequency response on the unit circle
w, h = signal.freqz_zpk(sys_z.zeros, sys_z.poles, sys_z.gain, worN=w_digital)

mag_db = 20*np.log10(np.maximum(np.abs(h), 1e-300))
phase_deg = np.unwrap(np.angle(h)) * 180/np.pi

fig, ax = plt.subplots(2, 1, figsize=(9, 7), sharex=True)

ax[0].semilogx(f, mag_db)
ax[0].axvline(400, linestyle='--', linewidth=1)
ax[0].set_ylabel("Magnitude (dB)")
ax[0].grid(True, which="both")

ax[1].semilogx(f, phase_deg)
ax[1].axvline(400, linestyle='--', linewidth=1)
ax[1].set_ylabel("Phase (deg)")
ax[1].set_xlabel("Frequency (Hz)")
ax[1].grid(True, which="both")

plt.tight_layout()
plt.show()

## 6. Impulse response (time domain)

For a discrete-time LTI system, the impulse response \(h[n]\) is the output when the input is \(\delta[n]\).

We compute it by filtering an impulse through the digital filter.

> This impulse response belongs to the **digital** system produced by the analog→digital mapping (so it depends on the mapping and on the sampling rate).


In [ ]:
# Convert ZPK to transfer function (b, a) for filtering
b, a = signal.zpk2tf(sys_z.zeros, sys_z.poles, sys_z.gain)

# Make an impulse
N = 2000
x = np.zeros(N)
x[0] = 1.0

# Filter it
h_imp = signal.lfilter(b, a, x)

t = np.arange(N) / fs

plt.figure(figsize=(9, 3.5))
plt.plot(t, h_imp)
plt.xlabel("Time (s)")
plt.ylabel("h[n]")
plt.title("Impulse response (digital system)")
plt.grid(True, which="both")
plt.tight_layout()
plt.show()

## 7. Optional: compare mappings (matched-z vs bilinear)

Try toggling:

```python
method = "matched_z"
# method = "bilinear"
```

and rerun sections 3–6.

Different transforms preserve different properties of the analog system, so you may see slightly different frequency/impulse responses.


## Summary

- **S-domain** poles/zeros describe an analog (continuous-time) transfer function \(H(s)\).
- You can map them to the **Z-domain** to study a discrete-time system \(H(z)\).
- The frequency response is found on the **unit circle** \(z=e^{i\omega}\).
- Magnitude is controlled by **geometry**:
  - product of distances to zeros in the numerator
  - product of distances to poles in the denominator
